# Offline policy evaluation

## Setup

In [1]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, roc_auc_score

import scipy
import scipy.stats as st

import matplotlib.pyplot as plt

## Data

In [2]:
# Read data with excess columns dropping
excess_data = ["campaign_clicks", "impressions"]
data = pd.read_csv("../task_1_recsys/data/data.csv").drop(excess_data, axis=1)
data.head()

,date_time,zone_id,banner_id,oaid_hash,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1


In [8]:
# Support function
def prepare_data(data: pd.DataFrame):
    
    # Drop nulls
    data.dropna(inplace=True)
    
    # Drop negative stds
    dindex = data[(data.g0 < 0) | (data.g1 < 0)].index
    data.drop(dindex, inplace=True)

    # Delete raws with banner_id != banner_id_0
    dindex = data[data.banner_id != data.banner_id0].index
    data.drop(dindex, inplace=True)
    data.drop(["banner_id0"], axis=1, inplace=True)

    # Datetime features
    data["date_time"] = pd.to_datetime(data["date_time"])
    data["hour"] = data["date_time"].dt.hour

    # Train test split
    last_day = data["date_time"].max()
    train = data[data["date_time"].dt.date < last_day.date()]
    test = data[data["date_time"].dt.date == last_day.date()]

    # Prepare features
    cols_to_del = ["clicks", "date_time", "banner_id1", "g0", "g1", "coeff_sum0",  "coeff_sum1"]
    X_train = train.drop(cols_to_del, axis=1, inplace=False)
    X_test = test.drop(cols_to_del, axis=1, inplace=False)

    # OHE
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=True)
    X_train = ohe.fit_transform(X_train)
    X_test = ohe.transform(X_test)

    # Prepare labels
    y_train = train["clicks"]
    y_test = test["clicks"]

    # Distribution
    dstrb = test[["g0", "g1", "coeff_sum0", "coeff_sum1"]]

    # Prepare test
    cols_to_del = ["clicks", "date_time", "g0", "g1", "coeff_sum0",  "coeff_sum1"]
    test.drop(cols_to_del, axis=1, inplace=True)

    # pi_0
    X_test_pi0 = test.drop(["banner_id1"], axis=1, inplace=False)
    X_test_pi0 = ohe.transform(X_test_pi0)

    # pi_1
    test["banner_id"] = test["banner_id1"]
    X_test_pi1 = test.drop(["banner_id1"], axis=1)
    X_test_pi1 = ohe.transform(X_test_pi1)

    return X_train, y_train, X_test, y_test, X_test_pi0, X_test_pi1, dstrb

In [9]:
X_train, y_train, X_test, y_test, X_test_pi0, X_test_pi1, dstrb = prepare_data(data.copy())

C:\Users\user\AppData\Local\Temp\ipykernel_7088\3936093563.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(cols_to_del, axis=1, inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_7088\3936093563.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["banner_id"] = test["banner_id1"]


## Model

Модель – линейная регрессия *(см. HW_1)*

In [10]:
def create_model(solver, C, penalty, random_state):
    model = LogisticRegression(solver=solver, C=C, penalty=penalty, random_state=random_state)
    return model

In [11]:
model = create_model(solver='liblinear', C=0.01, penalty="l2", random_state=42)
model.fit(X_train, y_train)

LogisticRegression(C=0.01, random_state=42, solver='liblinear')

In [13]:
y_baseline = np.mean(y_test)*np.ones(y_test.shape)
y_model = model.predict_proba(X_test)[:, 1]

print(f"Model: Log-loss = {log_loss(y_test, y_model)}, ROC-AUC = {roc_auc_score(y_test, y_model)}")
print(f"Baseline: Log-loss = {log_loss(y_test, y_baseline)}, ROC-AUC = {roc_auc_score(y_test, y_baseline)}")

Model: Log-loss = 0.13339720171424882, ROC-AUC = 0.7919427093037561
Baseline: Log-loss = 0.155164669729177, ROC-AUC = 0.5


In [14]:
def logit(x):
    return np.log(x) - np.log(1 - x)

In [16]:
coeff_sum_pi0 = logit(model.predict_proba(X_test_pi0)[:, 1])
coeff_sum_pi1 = logit(model.predict_proba(X_test_pi1)[:, 1])

## Clipped Inverse Propensity Score

/*Вставить пояснение с формулой расчёта*/

In [17]:
pi_0 = st.norm.cdf((dstrb.coeff_sum1 - dstrb.coeff_sum0) / np.sqrt(dstrb.g0 ** 2 + dstrb.g1 ** 2 + 1e-7))
pi_1 = st.norm.cdf((coeff_sum_pi1 - coeff_sum_pi0) / np.sqrt(dstrb.g0 ** 2 + dstrb.g1 ** 2 + 1e-7))

In [19]:
cips = np.mean(y_test * np.clip(pi_1/(pi_0 + 1e-7), 0, 10))
print(f"Clipped Inverse Propensity Score: {cips:.5f}")

Clipped Inverse Propensity Score: 0.08496
